# PMS

In [1]:
import threading
import numpy as np
import cv2
import pyrealsense2 as rs
import trimesh
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *

# Face Detector
class FaceDetector:
    def __init__(self, model_path, config_path):
        self.net = cv2.dnn.readNetFromTensorflow(model_path, config_path)
        self.pipeline = rs.pipeline()
        config = rs.config()
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
        self.pipeline.start(config)
        self.rotation_data = [0, 0]
        self.running = True
        self.movement_detected = False
        threading.Thread(target=self.process_frames, daemon=True).start()

    def process_frames(self):
        while self.running:
            frames = self.pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            if not color_frame:
                continue
            color_image = np.asanyarray(color_frame.get_data())
            h, w = color_image.shape[:2]

            blob = cv2.dnn.blobFromImage(color_image, 1.0, (300, 300), [104, 117, 123], swapRB=False)
            self.net.setInput(blob)
            detections = self.net.forward()

            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]
                if confidence > 0.7:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    startX, startY, endX, endY = box.astype("int")
                    
                    center_x = (startX + endX) // 2
                    center_y = (startY + endY) // 2

                    dx = (center_x - w // 2) / (w // 2)
                    dy = (center_y - h // 2) / (h // 2)

                    rot_x = np.clip(dy * -90, -90, 90)
                    rot_y = np.clip(dx * -90, -90, 90)

                    self.rotation_data = [rot_x, rot_y]
                    self.movement_detected = True
                    
                    cv2.rectangle(color_image, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    break

            cv2.imshow("Face Detector", color_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.running = False
                break

    def get_rotation(self):
        return self.rotation_data

    def has_detected_movement(self):
        return self.movement_detected

    def stop(self):
        self.running = False
        self.pipeline.stop()
        cv2.destroyAllWindows()


class HologramViewer:
    def __init__(self, model_path, face_detector=None):
        pygame.init()
        self.width, self.height = 800, 600
        pygame.display.set_mode((self.width, self.height), DOUBLEBUF | OPENGL)
        pygame.display.set_caption("Holographic 3D Model Viewer")

        glEnable(GL_DEPTH_TEST)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)

        glEnable(GL_LIGHTING)
        glEnable(GL_LIGHT0)
        glEnable(GL_COLOR_MATERIAL)
        glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)

        glLightfv(GL_LIGHT0, GL_POSITION, [0, 0, 10, 1])
        glLightfv(GL_LIGHT0, GL_AMBIENT, [0.1, 0.3, 0.5, 1])
        glLightfv(GL_LIGHT0, GL_DIFFUSE, [0.4, 0.6, 0.8, 1])
        glLightfv(GL_LIGHT0, GL_SPECULAR, [0.8, 0.8, 1.0, 1])

        self.model = trimesh.load(model_path)
        self.vertices = self.model.vertices
        self.faces = self.model.faces
        self.face_detector = face_detector

        self.model_position = [0, 0, 0]

        self.center_and_scale_model()

        self.hologram_alpha = 0.7
        self.hologram_glow = 0.3

        self.model_display_list = self.create_model_display_list()

        self.camera_distance = 5.0
        self.camera_elevation = 30
        self.camera_azimuth = -60

        self.setup_projection()

        self.running = True
        self.face_control_active = False  # Whether face movement has started control

    def center_and_scale_model(self):
        mins = np.min(self.vertices, axis=0)
        maxs = np.max(self.vertices, axis=0)
        dimensions = maxs - mins
        center = (mins + maxs) / 2
        max_dim = np.max(dimensions)
        scale_factor = 2.0 / max_dim if max_dim > 0 else 1.0

        for i in range(len(self.vertices)):
            self.vertices[i] = (self.vertices[i] - center) * scale_factor + np.array(self.model_position)

    def create_model_display_list(self):
        display_list = glGenLists(1)
        glNewList(display_list, GL_COMPILE)
        glColor4f(0.3, 0.7, 1.0, self.hologram_alpha)
        glBegin(GL_TRIANGLES)
        for face in self.faces:
            v0, v1, v2 = self.vertices[face[0]], self.vertices[face[1]], self.vertices[face[2]]
            normal = np.cross(v1 - v0, v2 - v0)
            if np.linalg.norm(normal) > 0:
                normal = normal / np.linalg.norm(normal)
            glNormal3fv(normal)
            glVertex3fv(v0)
            glVertex3fv(v1)
            glVertex3fv(v2)
        glEnd()
        glEndList()
        return display_list

    def setup_projection(self):
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluPerspective(45, (self.width / self.height), 0.1, 50.0)

    def update_camera(self):
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        phi = np.radians(90 - self.camera_elevation)
        theta = np.radians(self.camera_azimuth)
        x = self.model_position[0] + self.camera_distance * np.sin(phi) * np.cos(theta)
        y = self.model_position[1] + self.camera_distance * np.sin(phi) * np.sin(theta)
        z = self.model_position[2] + self.camera_distance * np.cos(phi)
        gluLookAt(x, y, z, *self.model_position, 0, 0, 1)

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
                self.running = False

    def update_from_face_detector(self):
        if self.face_detector and self.face_detector.has_detected_movement():
            self.face_control_active = True
        if self.face_control_active:
            rot_x, rot_y = self.face_detector.get_rotation()
            self.camera_elevation = np.clip(-rot_x, -85, 85)
            self.camera_azimuth = np.clip(-rot_y * 2, -90, 90)

    def draw_hologram_glow(self):
        glDisable(GL_LIGHTING)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE)
        glPushMatrix()
        glScalef(1.05, 1.05, 1.05)
        glColor4f(0.3, 0.7, 1.0, self.hologram_glow)
        glCallList(self.model_display_list)
        glPopMatrix()
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        glEnable(GL_LIGHTING)

    def draw(self):
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glClearColor(0.05, 0.05, 0.1, 1.0)
        self.update_camera()
        glPushMatrix()
        glCallList(self.model_display_list)
        self.draw_hologram_glow()
        glPopMatrix()
        pygame.display.flip()

    def run(self):
        clock = pygame.time.Clock()
        while self.running:
            self.handle_events()
            self.update_from_face_detector()
            self.draw()
            clock.tick(60)


def main():
    print("Holographic 3D Model Viewer")
    try:
        model_path = r"C:\Users\junio\Downloads\GitHub\IA\PM\3D in blender\rp_dennis_posed_004_30k.OBJ"
        detector = FaceDetector(
            r"C:\Users\junio\Downloads\GitHub\IA\PM\FD\opencv_face_detector_uint8.pb",
            r"C:\Users\junio\Downloads\GitHub\IA\PM\FD\opencv_face_detector.pbtxt"
        )
        viewer = HologramViewer(model_path, detector)
        viewer.run()
    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        if 'detector' in locals():
            detector.stop()
        pygame.quit()
        print("Projection closed!")


if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
Holographic 3D Model Viewer
Projection closed!
